[Audio Data](https://drive.google.com/drive/folders/1802iY9TDPscgavH7Alz_MxjznL8tmV8O) | [whisper GitHub](https://github.com/openai/whisper/blob/main/README.md)

In [47]:
!pip install --upgrade pip setuptools wheel


In [48]:
!pip install openai-whisper --no-deps


In [49]:
!pip install numpy tqdm tiktoken more-itertools


In [50]:
!pip install numba

In [51]:
!pip install torch --index-url https://download.pytorch.org/whl/cpu/


Looking in indexes: https://download.pytorch.org/whl/cpu/


In [52]:
!pip install triton>=2.0.0

In [53]:
!pip install ffmpeg

In [54]:
import whisper
print(whisper.__version__)


20240930


In [58]:
import whisper
import torch

# Load the tiny model
model = whisper.load_model("base")



/home/ferganey/spechmdl/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [59]:
# Check if the model is on the CPU
device = torch.device("cpu")
model.to(device)

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-5): 6 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=512, out_features=512, bias=True)
          (key): Linear(in_features=512, out_features=512, bias=False)
          (value): Linear(in_features=512, out_features=512, bias=True)
          (out): Linear(in_features=512, out_features=512, bias=True)
        )
        (attn_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (mlp_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((512,), eps=1e-05,

In [60]:

# Transcribe an audio file
result = model.transcribe("/media/ferganey/Data1/03_Projects/Final_Learning_AI-Projects/02_DeepLearning_Projects/00_Speech_Recognition_RasPi_ContainerzedApp/01_Dataset/00_Audio/shopping.mp3")

# Print the transcribed text
print(result["text"])

/home/ferganey/spechmdl/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 shopping at the mall. At one time in North America most people shopped downtown on Main Street. Most businesses were at the center of town. When people started using automobiles however they moved away from downtown. In time most people lived in the suburbs. Eventually stores and small shopping plazas were built in suburban areas. Still most of the big stores were downtown. But as more and more cars were on the roads driving and parking downtown became a problem. There wasn't room for a lot of cars to park downtown. People also didn't want to fight downtown traffic just to go shopping. So in the 1950s and 1960s there was the beginning of large suburban shopping malls and plazas. Plazas were a row of stores attached to one another. Malls were usually a double row of stores with a roof connecting both rows. This means that shoppers did all their shopping inside. Large department stores and grocery stores were usually part of the mall but there were many smaller stores as well. When you 

In [61]:

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("/media/ferganey/Data1/03_Projects/Final_Learning_AI-Projects/02_DeepLearning_Projects/00_Speech_Recognition_RasPi_ContainerzedApp/01_Dataset/00_Audio/shopping.mp3")


In [62]:
# Only take the first 30 seconds with the correct sampling rate (16000 Hz)
max_duration = 30  # 30 seconds
sample_rate = 16000  # Whisper's default sampling rate
audio = audio[:max_duration * sample_rate]  # Slice the first 30 seconds
audio = whisper.pad_or_trim(audio)

# Generate log-Mel spectrogram
mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)

# Detect the language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# Decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# Output the result
print(result.text)

Detected language: en
shopping at the mall. At one time in North America most people shopped downtown on Main Street. Most businesses were at the center of town. When people started using automobiles however they moved away from downtown. In time most people lived in the suburbs. Eventually stores and small shopping plazas were built in suburban areas. Still most of the big stores were downtown. But as more and more cars were on the roads driving and parking downtown became a problem. There


In [63]:

# Calculate the size in bytes
total_params = sum(p.numel() for p in model.parameters())  # Total number of parameters
total_size_bytes = total_params * 4  # Assuming 4 bytes per parameter (float32)

# Convert bytes to megabytes
total_size_mb = total_size_bytes / (1024 ** 2)

print(f"Total Parameters: {total_params}")
print(f"Total Model Size: {total_size_mb:.2f} MB")

Total Parameters: 71825920
Total Model Size: 273.99 MB


In [64]:
print(model)


Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-5): 6 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=512, out_features=512, bias=True)
          (key): Linear(in_features=512, out_features=512, bias=False)
          (value): Linear(in_features=512, out_features=512, bias=True)
          (out): Linear(in_features=512, out_features=512, bias=True)
        )
        (attn_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (mlp_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((512,), eps=1e-05,

In [ ]:

# Verify all model weights are float32
all([param.dtype == torch.float32 for name, param in model.named_parameters()])


True